In [2]:
%reload_ext autoreload
%autoreload 2

In [9]:
import os
import sys

curruser = os.environ.get('USER')
sys.path.insert(0, './src/')
# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
sys.path.insert(0, '/home/{}/python36-libs/lib/python3.6/site-packages/'.format(curruser))
from pathlib import Path
import re
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
# import utils
from LSTMPreProc import preprocIO
from Bio import SeqIO

# from bpe import Encoder
from sklearn.preprocessing import LabelEncoder
import sentencepiece as spm
# from tokenizers import CharBPETokenizer, ByteLevelBPETokenizer, SentencePieceBPETokenizer

In [3]:
dnaSegPath = ('./data/dnaseg/mart_export.txt')
prep = preprocIO()
seqArr = []
for seq in tqdm_notebook(prep.streamFastaIO(dnaSegPath)):
     seqArr+=[seq]

In [ ]:
with open('./data/dnaseg/mart_export_proc.txt', 'w+') as fout:
    fout.write('\n'.join(seqArr))

In [27]:
seqArr = []
with open('./data/dnaseg/mart_export_proc.txt', mode='r', newline='\n') as fout:
    lines = fout.readlines()
    for line in tqdm_notebook(lines):
        seqArr += [line.split('\n')[0]]
    fout.close()

In [6]:
_str = 'CTACCACCCTCCTCCTCGCATGCCACCTCCATTAGAGGTCGGGGTCGTGGTGGGGGGAGA'

In [30]:
bpe = Encoder(    
    vocab_size=400,
    pct_bpe=0.9,
    word_tokenizer=None,
    silent=True,
    ngram_min=1,
    ngram_max=10,)

In [31]:
bpe.fit(seqArr)

In [32]:
bpe.save("models/BPE/bpe.model")

In [55]:
bpe = Encoder.load("models/BPE/bpe.model")

In [56]:
bpe.vocab_size

400

In [53]:
bpe_dct = bpe.learn_bpe_vocab([_str])
bpe_dct.pop('__sow')
bpe_dct.pop('__eow')
idxs = [v for k,v in bpe_dct.items()]

In [57]:
len(idxs)

359

In [ ]:
spt = SentencePieceBPETokenizer()

In [15]:
spt.train('data/dnaseg/mart_export_proc.txt', vocab_size = 300,)

In [16]:
enc = spt.encode_tokenized([_str])

In [17]:
enc.tokens

['C',
 'TACC',
 'ACCC',
 'TCC',
 'TCCTC',
 'GC',
 'ATGCC',
 'ACCTCC',
 'ATT',
 'AGAGG',
 'TC',
 'GGGG',
 'TCG',
 'TGG',
 'TGGG',
 'GGG',
 'AG',
 'A']

In [3]:
byteBPE = ByteLevelBPETokenizer()

In [5]:
byteBPE.train('data/dnaseg/mart_export_proc.txt', vocab_size = 300,)

In [13]:
byteBPE.save("models/BPE","bytebpe.model")

['models/BPE/bytebpe.model-vocab.json', 'models/BPE/bytebpe.model-merges.txt']

In [9]:
enc = byteBPE.encode_tokenized([_str])

In [11]:
enc.ids

[34,
 51,
 266,
 296,
 267,
 267,
 260,
 276,
 272,
 258,
 266,
 267,
 278,
 256,
 269,
 260,
 262,
 262,
 260,
 38,
 264,
 288,
 262,
 38,
 256,
 32]

In [15]:
spm.SentencePieceTrainer.Train('--input=data/dnaseg/mart_export_proc.txt --model_prefix=unigram --vocab_size=500 --model_type=unigram')


True

In [5]:
sp = spm.SentencePieceProcessor()
sp.Load("models/BPE/sentpbpe.model")

True

In [7]:
sp.EncodeAsPieces(_str)

['▁',
 'C',
 'TACC',
 'ACCC',
 'TCC',
 'TCCTC',
 'GC',
 'ATGCC',
 'ACC',
 'TCC',
 'ATT',
 'AGAGG',
 'TC',
 'GGGG',
 'TCG',
 'TGG',
 'TGGGG',
 'GG',
 'AG',
 'A']

In [74]:
sp = spm.SentencePieceProcessor()
sp.Load("models/BPE/unigram.model")

True

In [75]:
sp.GetPieceSize()

500

In [77]:
sp.EncodeAsPieces(_str)

['▁',
 'CTAC',
 'CACC',
 'CTCC',
 'TCCTC',
 'GCA',
 'TGCCA',
 'CCTCC',
 'ATTA',
 'GAGG',
 'TCGG',
 'GGTC',
 'GTGGTG',
 'GGG',
 'GGAGA']

In [103]:
sp.EncodeAsIds(_str)

[499, 100, 77, 71, 252, 65, 247, 275, 277, 64, 326, 363, 424, 23, 361]

In [100]:
sp.IdToPiece(0)

'<unk>'

In [102]:
for x in range(5):
    res = sp.SampleEncodeAsIds(_str, -1, 0.01)
    print(res, len(res))

[499, 7, 30, 5, 69, 29, 30, 7, 7, 37, 30, 43, 83, 7, 7, 11, 7, 37, 33, 5, 9, 5, 105, 43, 23, 17, 9, 30, 9, 8, 9, 9, 20, 14, 9, 5] 36
[499, 118, 7, 7, 11, 7, 28, 7, 30, 7, 28, 12, 16, 12, 7, 11, 28, 7, 33, 125, 9, 115, 9, 363, 8, 9, 30, 20, 9, 23, 5, 9, 5] 33
[499, 7, 30, 5, 7, 7, 11, 7, 71, 30, 7, 7, 30, 7, 12, 5, 30, 12, 31, 7, 37, 16, 30, 5, 9, 5, 9, 9, 30, 7, 9, 20, 9, 30, 7, 8, 45, 9, 20, 9, 9, 9, 3, 5] 44
[499, 7, 19, 31, 7, 28, 71, 30, 7, 9, 56, 12, 7, 5, 7, 28, 56, 38, 14, 9, 9, 115, 9, 105, 183, 9, 8, 20, 23, 5, 9, 5] 32
[499, 7, 38, 7, 77, 7, 30, 7, 7, 30, 7, 280, 7, 16, 9, 7, 31, 7, 30, 69, 25, 5, 9, 5, 20, 17, 9, 9, 363, 45, 9, 45, 20, 20, 9, 14, 14] 37


In [112]:
for x in range(5):
    res = sp.SampleEncodeAsPieces(_str, -1, 0.01)
    print(res, len(res))

['▁', 'C', 'T', 'A', 'CCA', 'CCC', 'TCC', 'T', 'CCTC', 'GC', 'A', 'T', 'GCC', 'ACC', 'TC', 'C', 'A', 'TTAG', 'A', 'GG', 'T', 'CG', 'GG', 'G', 'T', 'CGTG', 'GTGG', 'GG', 'G', 'G', 'A', 'G', 'A'] 33
['▁', 'CTA', 'C', 'C', 'AC', 'C', 'C', 'TCC', 'T', 'CCT', 'CGC', 'A', 'T', 'GC', 'CAC', 'CTC', 'C', 'A', 'T', 'T', 'AGAG', 'GT', 'CG', 'G', 'G', 'GT', 'CG', 'T', 'GG', 'T', 'G', 'GG', 'G', 'GGAGA'] 34
['▁', 'C', 'T', 'AC', 'C', 'AC', 'C', 'CTCC', 'TC', 'C', 'TC', 'G', 'C', 'A', 'TGC', 'C', 'ACCT', 'CCA', 'T', 'T', 'A', 'G', 'AG', 'GTC', 'GGG', 'G', 'T', 'CG', 'TGG', 'T', 'G', 'G', 'GG', 'GG', 'A', 'GA'] 36
['▁', 'C', 'T', 'A', 'C', 'CACC', 'CTCC', 'T', 'CCT', 'C', 'G', 'CAT', 'G', 'CCA', 'CCT', 'CCA', 'T', 'T', 'AG', 'A', 'G', 'GT', 'CG', 'GGG', 'TC', 'GTG', 'GTG', 'GG', 'GGG', 'AG', 'A'] 31
['▁', 'C', 'T', 'A', 'CCAC', 'C', 'C', 'TC', 'C', 'T', 'C', 'C', 'T', 'CG', 'C', 'AT', 'GC', 'C', 'ACC', 'T', 'C', 'C', 'AT', 'T', 'A', 'GAGG', 'T', 'C', 'G', 'G', 'G', 'G', 'TC', 'GT', 'GGT', 'G', 'GG', 